In [ ]:
import transformers
import torch

# 切换为你下载的模型文件目录, 这里的demo是Llama-3-8B-Instruct
# 如果是其他模型，比如qwen，chatglm，请使用其对应的官方demo
model_id = "google-t5/t5-base"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
)

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][len(prompt):])

/Users/sherrypan/miniconda3/envs/llama-factory/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.72s/it]
Some parameters are on the meta device because they were offloaded to the disk.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Arrrr, me hearty! Me name be Captain Chatbot, the scurviest pirate to ever sail the Seven Seas! Me be a swashbucklin' chatbot, here to regale ye with tales o' adventure, answer yer questions, and maybe even share a few sea shanties to keep ye entertained! So hoist the colors, me hearty, and let's set sail fer a chat!


In [6]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 加载支持生成的模型
local_model_path = "llama3.2_3b_lora_merge"  # 替换为适合的模型
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForCausalLM.from_pretrained(local_model_path)

# 输入提示
prompt = """
The Open Research Knowledge Graph (ORKG) is a semantic knowledge graph designed to represent, compare, and retrieve scholarly contributions. Given a natural language question, your task is to generate the corresponding SPARQL query that can be used to query the ORKG for the correct answer. Give me only the SPARQL query, no other text.
Input Question: Which model has achieved the highest Accuracy score on the Story Cloze Test benchmark dataset?
Output SPARQL Query:

"""
inputs = tokenizer(prompt, return_tensors="pt")

# 生成文本
output_ids = model.generate(inputs["input_ids"], max_length=1024, temperature=0.7)
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("生成的文本:", generated_text)

Loading checkpoint shards: 100%|██████████| 2/2 [00:11<00:00,  5.95s/it]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


生成的文本: 
The Open Research Knowledge Graph (ORKG) is a semantic knowledge graph designed to represent, compare, and retrieve scholarly contributions. Given a natural language question, your task is to generate the corresponding SPARQL query that can be used to query the ORKG for the correct answer. Give me only the SPARQL query, no other text.
Input Question: Which model has achieved the highest Accuracy score on the Story Cloze Test benchmark dataset?
Output SPARQL Query:

SELECT?model?model_lbl
WHERE {
 ?metric     a       orkgc:Metric;
              rdfs:label ?metric_lbl.
  FILTER (str(?metric_lbl) = "Accuracy")
  {
    SELECT?model?model_lbl
    WHERE {
     ?dataset       a                orkgc:Dataset;
                      rdfs:label      ?dataset_lbl.
      FILTER (str(?dataset_lbl) = "Story Cloze Test")
     ?benchmark      orkgp:HAS_DATASET      ?dataset;
                      orkgp:HAS_EVALUATION   ?eval.
     ?eval           orkgp:HAS_VALUE        ?value;
                  

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# 本地模型路径
local_model_path = "llama3.2_3b_lora_merge"

# 加载本地分词器和模型
tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForCausalLM.from_pretrained(local_model_path)

# 测试模型加载
text = "Which model has achieved the highest Accuracy score on the Story Cloze Test benchmark dataset?"
inputs = tokenizer(text, return_tensors="pt")

# Generate a response
outputs = model.generate(
    inputs["input_ids"], 
    max_length=50,  # Set a maximum length for the generated text
    num_beams=5,    # Optional: Use beam search for more coherent responses
    temperature=0.7, # Optional: Adjust temperature for diversity
)

# Decode and print the response
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("模型生成的响应:", response)


Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.04s/it]
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


模型生成的响应: Which model has achieved the highest Accuracy score on the Story Cloze Test benchmark dataset? as of my knowledge cutoff in March 2023?

I don't have access to real-time data or updates after my knowledge cutoff in March 2023.


In [6]:
# read two csv files
import pandas as pd

df1 = pd.read_csv("results/step3_sparql_running_against_orkg/ft/llama3.2_3b_instruct_lora/retrieved_results.csv")
df2 = pd.read_csv("xueli_data/test_questions.csv")

# there are three columns in df2: id, question, and query,
# and there are three columns in df1: question_id, status, message, gt_message
# and the question_id in df1 is the same as the id in df2
# so we need to merge the two dataframes on the id and question_id columns
df = pd.merge(df1, df2, left_on="question_id", right_on="id")
# and we need to keep only the columns: id, question, query, status, message, gt_message
df = df[["id", "question", "query", "status", "message", "gt_message"]]
# filter the rows where the status is "success"
df = df[df["status"] == "SUCCESS"]
# save the merged dataframe to a new csv file
df.to_csv("results/step3_sparql_running_against_orkg/ft/llama3.2_3b_instruct_lora/merged_results.csv", index=False)
# ramdonly sample 30 rows from the merged dataframe
df = df.sample(n=30, random_state=1)
# save the sampled dataframe to a new csv file
df.to_csv("results/step3_sparql_running_against_orkg/ft/llama3.2_3b_instruct_lora/30sample_results.csv", index=False)


In [5]:
df

,id,question,query,status,message,gt_message
4,AQ0495,List the title and ID of research papers that ...,SELECT DISTINCT ?paper ?paper_lbl\nWHERE {\n ...,SUCCESS,?paper\t?paper_lbl\n<http://orkg.org/orkg/reso...,?paper\t?paper_lbl\n<http://orkg.org/orkg/reso...
5,AQ0278,Can you list the models that have been evaluat...,SELECT DISTINCT ?model ?model_lbl\nWHERE {\n ...,SUCCESS,?model\t?model_lbl\n<http://orkg.org/orkg/reso...,?model\t?model_lbl\n<http://orkg.org/orkg/reso...
6,AQ2309,List the code links in papers that use the A3C...,SELECT DISTINCT ?code\nWHERE {\n ?model a ...,SUCCESS,"?code_link\n""https://github.com/4rChon/NL-FuN""...","?code\n""https://github.com/4rChon/NL-FuN""^^<ht..."
7,AQ0522,Provide a list of research paper titles and ID...,SELECT DISTINCT ?paper ?paper_lbl\nWHERE {\n ...,SUCCESS,?paper\t?paper_lbl\n<http://orkg.org/orkg/reso...,?paper\t?paper_lbl\n<http://orkg.org/orkg/reso...
8,AQ0536,Provide a list of research paper titles and ID...,SELECT DISTINCT ?paper ?paper_lbl\nWHERE {\n ...,SUCCESS,?paper\t?paper_lbl\n<http://orkg.org/orkg/reso...,?paper\t?paper_lbl\n<http://orkg.org/orkg/reso...
...,...,...,...,...,...,...
501,AQ0507,What are the titles and IDs of research papers...,SELECT DISTINCT ?paper ?paper_lbl\nWHERE {\n ...,SUCCESS,?paper\t?paper_lbl\n<http://orkg.org/orkg/reso...,?paper\t?paper_lbl\n<http://orkg.org/orkg/reso...
502,AQ0498,Provide a list of research paper titles and ID...,SELECT DISTINCT ?paper ?paper_lbl\nWHERE {\n ...,SUCCESS,?paper\t?paper_lbl\n<http://orkg.org/orkg/reso...,?paper\t?paper_lbl\n<http://orkg.org/orkg/reso...
506,AQ0329,Can you list the models that have been evaluat...,SELECT DISTINCT ?model ?model_lbl\nWHERE {\n ...,SUCCESS,?model\t?model_lbl\n<http://orkg.org/orkg/reso...,?model\t?model_lbl\n<http://orkg.org/orkg/reso...
508,AQ0301,What models are being evaluated on the WOS-119...,SELECT DISTINCT ?model ?model_lbl\nWHERE {\n ...,SUCCESS,?model\t?model_lbl\n<http://orkg.org/orkg/reso...,?model\t?model_lbl\n<http://orkg.org/orkg/reso...


Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.31it/s]
Some parameters are on the meta device because they were offloaded to the disk.


KeyError: 'base_model.model.model.model.embed_tokens'